In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np
import os

from tensorflow.keras.models import save_model


In [3]:
# Directories
train_data_dir = '/content/drive/MyDrive/preprocessed_paddies/train'
validation_data_dir = '/content/drive/MyDrive/preprocessed_paddies/validate'
test_data_dir = '/content/drive/MyDrive/preprocessed_paddies/test'

# Data generators for loading the dataset
train_datagen = image.ImageDataGenerator(rescale=1./255)
validation_datagen = image.ImageDataGenerator(rescale=1./255)
test_datagen = image.ImageDataGenerator(rescale=1./255)

# Load datasets without augmentation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(500, 250),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(500, 250),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(500, 250),
    batch_size=32,
    class_mode='categorical'
)


Found 8813 images belonging to 10 classes.
Found 700 images belonging to 10 classes.
Found 1300 images belonging to 10 classes.


In [4]:
# Model architecture with Dropout and L2 Regularization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(500, 250, 3), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Learning rate scheduler function
def lr_schedule(epoch):
    lr = 0.001
    if epoch > 50:
        lr *= 0.1
    elif epoch > 30:
        lr *= 0.5
    return lr

# Compile the model with a learning rate scheduler
optimizer = Adam()
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Include the learning rate scheduler in the callbacks
callbacks = [LearningRateScheduler(lr_schedule, verbose=1)]

# Train the model
history = model.fit(train_generator, epochs=60, validation_data=validation_generator, callbacks=callbacks)


!pip install pyyaml h5py

model.save('/content/drive/MyDrive/Colab Notebooks/paddy_model_one.keras')
model.save('/content/drive/MyDrive/Colab Notebooks/paddy_model_one.h5')
model.save('paddy_model_two.keras')
model.save('paddy_model_two.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/60
276/276 ━━━━━━━━━━━━━━━━━━━━ 4647s 17s/step - accuracy: 0.6398 - loss: 6.5231 - val_accuracy: 0.1000 - val_loss: 66.5515 - learning_rate: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/60
276/276 ━━━━━━━━━━━━━━━━━━━━ 68s 247ms/step - accuracy: 0.8343 - loss: 2.0766 - val_accuracy: 0.4329 - val_loss: 12.5677 - learning_rate: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/60
276/276 ━━━━━━━━━━━━━━━━━━━━ 81s 245ms/step - accuracy: 0.8810 - loss: 1.5300 - val_accuracy: 0.3000 - val_loss: 28.9558 - learning_rate: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/60
276/276 ━━━━━━━━━━━━━━━━━━━━ 68s 247ms/step - accuracy: 0.9021 - loss: 1.2370 - val_accuracy: 0.6914 - val_loss: 2.7883 - learning_rate: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/60
276/276 ━━━━━━━━━━━━━━━━━━━━ 81s 244ms/step - 

In [ ]:
!pip install pyyaml h5py

In [ ]:
# model.save('model.h5')
model.save('my_hello_model.keras')

In [ ]:
from google.colab import files

In [ ]:
files.download('model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy*100:.2f}%')

41/41 [==============================] - 761s 19s/step - loss: 0.4927 - accuracy: 0.9369
Test Accuracy: 93.69%


In [ ]:
directory_path = '/content/drive/MyDrive/ModelTraining/preprocessed_paddies/predict'
# List all files in the specified directory
files = os.listdir(directory_path)

# Iterate through the files and predict
for file in files:
    image_path = os.path.join(directory_path, file)

    if os.path.isfile(image_path):
        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(500, 250))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add an extra dimension for batch
        img_array /= 255.0  # Normalize pixel values

        # Make a prediction
        prediction = model.predict(img_array)
        print(f"Image: {file}")
        print(f"Prediction Array: {prediction}")

        # Get the predicted class index and confidence score
        predicted_class_index = np.argmax(prediction)
        confidence_score = prediction[0, predicted_class_index]

        # Define class labels
        class_labels = list(validation_generator.class_indices.keys())

        # Get the corresponding class label
        predicted_class_label = class_labels[predicted_class_index]

        # Display the results
        print(f'Predicted Class: {predicted_class_label}')
        print(f'Confidence Score: {confidence_score * 100:.2f}%')


1/1 [==============================] - 1s 637ms/step
Image: 362 (1).jpeg
Prediction Array: [[1.5889396e-05 9.9998415e-01 1.0628810e-11 3.7361032e-13 2.6187884e-13
  1.2614057e-15 6.7037154e-11 6.6892455e-14 7.4628297e-19 3.9806514e-09]]
Predicted Class: At 362
Confidence Score: 100.00%
1/1 [==============================] - 0s 17ms/step
Image: md (1).jpeg
Prediction Array: [[2.26420778e-18 4.32060393e-13 1.10683946e-16 6.32299211e-15
  7.94404542e-10 5.28779545e-15 1.70372715e-14 8.26221579e-18
  1.92277888e-13 1.00000000e+00]]
Predicted Class: Madathawalu
Confidence Score: 100.00%
1/1 [==============================] - 0s 18ms/step
Image: 352 (1).jpeg
Prediction Array: [[1.1551457e-10 5.9777302e-13 8.1712354e-15 3.3654701e-11 9.9999976e-01
  2.9189147e-07 3.0443587e-08 8.9223295e-10 2.7009877e-08 2.8304097e-09]]
Predicted Class: Bg 352
Confidence Score: 100.00%
1/1 [==============================] - 0s 18ms/step
Image: 359 (3).jpeg
Prediction Array: [[1.9641008e-10 9.9987236e-09 3.799

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('/content/model.h5')

In [ ]:
directory_path = '/content/drive/MyDrive/preprocessed_paddies/predict'
# List all files in the specified directory
files = os.listdir(directory_path)

# Iterate through the files and predict
for file in files:
    image_path = os.path.join(directory_path, file)

    if os.path.isfile(image_path):  # Ensure the path is a file to avoid errors
        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(500, 250))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add an extra dimension for batch
        img_array /= 255.0  # Normalize pixel values

        # Make a prediction
        prediction = model.predict(img_array)
        print(f"Image: {file}")  # Print the image name
        print(f"Prediction Array: {prediction}")

        # Get the predicted class index and confidence score
        predicted_class_index = np.argmax(prediction)
        confidence_score = prediction[0, predicted_class_index]

        # Define your class labels (assuming class labels are already retrieved from validation_generator)
        class_labels = list(validation_generator.class_indices.keys())

        # Get the corresponding class label
        predicted_class_label = class_labels[predicted_class_index]

        # Display the results
        print(f'Predicted Class: {predicted_class_label}')
        print(f'Confidence Score: {confidence_score * 100:.2f}%')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step
Image: 300 (2).JPEG
Prediction Array: [[0.09541062 0.10285097 0.10132349 0.10044992 0.10000096 0.1042571
  0.1019335  0.09561045 0.09552523 0.10263774]]


NameError: name 'validation_generator' is not defined

In [ ]:
directory_path = '/content/drive/MyDrive/ClusterTestPics/output'
# List all files in the specified directory
files = os.listdir(directory_path)

# Iterate through the files and predict
for file in files:
    image_path = os.path.join(directory_path, file)

    if os.path.isfile(image_path):  # Ensure the path is a file to avoid errors
        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(500, 250))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add an extra dimension for batch
        img_array /= 255.0  # Normalize pixel values

        # Make a prediction
        prediction = model.predict(img_array)
        print(f"Image: {file}")  # Print the image name
        print(f"Prediction Array: {prediction}")

        # Get the predicted class index and confidence score
        predicted_class_index = np.argmax(prediction)
        confidence_score = prediction[0, predicted_class_index]

        # Define your class labels (assuming class labels are already retrieved from validation_generator)
        class_labels = list(validation_generator.class_indices.keys())

        # Get the corresponding class label
        predicted_class_label = class_labels[predicted_class_index]

        # Display the results
        print(f'Predicted Class: {predicted_class_label}')
        print(f'Confidence Score: {confidence_score * 100:.2f}%')


1/1 [==============================] - 0s 18ms/step
Image: cropped_major_representative.jpg
Prediction Array: [[0.06817465 0.00343198 0.00199101 0.01556756 0.8457075  0.03086922
  0.0110274  0.0031779  0.0128036  0.00724916]]
Predicted Class: Bg 352
Confidence Score: 84.57%
1/1 [==============================] - 0s 18ms/step
Image: cropped_major_representative 3.jpg
Prediction Array: [[0.18165009 0.00696466 0.00104701 0.11529155 0.6510115  0.02447202
  0.0051887  0.00156298 0.00655316 0.00625826]]
Predicted Class: Bg 352
Confidence Score: 65.10%
1/1 [==============================] - 0s 17ms/step
Image: cropped_major_representative 5.jpg
Prediction Array: [[3.7893739e-01 4.4372723e-06 4.5242157e-08 2.6895970e-04 6.2077510e-01
  1.0018122e-05 1.1708942e-07 1.7826035e-07 2.5770264e-06 1.2159413e-06]]
Predicted Class: Bg 352
Confidence Score: 62.08%
1/1 [==============================] - 0s 17ms/step
Image: cropped_major_representative 7.jpg
Prediction Array: [[1.5581504e-01 2.9616070e-04